In [1]:

#using Pkg
#Pkg.add("CSV")
#Pkg.add("DataFrames")
#Pkg.add("JuMP")
#Pkg.add("Gurobi")



In [2]:
using CSV, DataFrames, JuMP, Gurobi

In [3]:
energy = CSV.read("energy.csv", DataFrame)
@show nrow(energy)
#energy

nrow(energy) = 1000


1000

In [4]:
utilization = CSV.read("utilization.csv", DataFrame)
@show nrow(utilization)
utilization

nrow(utilization) = 1000


Row,job,Machine 1,Machine 2,Machine 3,Machine 4,Machine 5,Machine 6,Machine 7,Machine 8,Machine 9,Machine 10,Machine 11,Machine 12,Machine 13,Machine 14,Machine 15,Machine 16,Machine 17,Machine 18,Machine 19,Machine 20
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,71.3739,118.118,66.416,104.537,51.1439,246.602,24.4154,117.744,170.301,133.563,102.631,32.8853,103.993,103.473,345.155,239.617,126.358,90.1242,110.464,249.585
2,2,87.0722,95.6556,158.638,199.409,416.969,72.8694,76.3235,265.946,135.55,62.4603,137.976,182.345,72.0339,18.6518,128.79,114.771,131.719,54.4402,198.57,182.801
3,3,85.8522,73.5966,78.6082,87.0531,89.4406,45.0581,58.9641,117.088,48.4202,91.0079,36.0306,124.496,98.7552,56.7721,51.2527,187.028,54.573,66.3453,200.062,152.656
4,4,106.931,77.9327,359.746,101.948,135.661,70.4738,120.757,57.1826,82.1369,108.988,170.512,160.123,189.812,136.883,178.749,47.5874,87.999,50.159,80.7611,122.512
5,5,62.1045,79.2495,84.1787,162.798,300.261,58.8958,63.14,119.338,119.873,148.778,159.413,127.768,60.2471,217.937,77.5972,95.1133,161.025,77.65,350.013,101.796
6,6,45.1771,52.8014,27.6889,47.4956,43.7341,77.0479,13.563,18.8494,65.3876,42.3736,80.3911,101.453,44.0569,17.7279,47.4617,25.1759,76.0424,24.8448,27.3681,28.364
7,7,121.477,107.507,90.5803,76.5274,140.806,20.7735,65.6033,18.2634,68.5753,142.525,91.771,216.837,46.6295,48.1777,111.718,162.235,96.217,37.325,46.2384,52.9135
8,8,58.2984,63.9953,32.4621,33.1899,84.9317,86.2332,79.0808,30.423,56.8788,48.6303,85.2619,36.2572,71.5648,15.4097,36.8312,49.507,33.5751,10.8508,54.439,76.9715
9,9,376.803,351.246,125.95,83.358,125.621,226.953,155.472,396.847,246.875,432.367,290.525,156.75,130.816,265.22,252.238,272.874,123.749,639.338,124.034,433.208


In [5]:
maxutil = CSV.read("maxutil.csv", DataFrame)
@show nrow(maxutil)
maxutil

nrow(maxutil) = 20


Row,machine,maxutil
,Int64,Float64
1,1,1170.65
2,2,1356.73
3,3,774.641
4,4,583.42
5,5,806.362
6,6,327.143
7,7,3895.12
8,8,591.396
9,9,1968.63


In [6]:
Mmaxutil = Matrix(maxutil);
Mmaxutil = Mmaxutil[:,2]


20-element Vector{Float64}:
 1170.6530507150087
 1356.7292590989266
  774.6407399067323
  583.42002453427
  806.3621979348665
  327.14308351538904
 3895.1172973703497
  591.3964093287686
 1968.62736382538
  376.4551601091232
 1964.1195955088235
  754.0803116165797
 3820.865117789083
  317.8379429234617
 1939.650469596368
  714.5178536320536
 1460.1424978957966
 1986.400958382396
  290.124209071122
 4947.224624995945

In [7]:
Mutilization = Matrix(utilization);
Mutilization = Mutilization[:,2:21]

1000×20 Matrix{Float64}:
  71.3739   118.118    66.416   104.537    …   90.1242  110.464   249.585
  87.0722    95.6556  158.638   199.409        54.4402  198.57    182.801
  85.8522    73.5966   78.6082   87.0531       66.3453  200.062   152.656
 106.931     77.9327  359.746   101.948        50.159    80.7611  122.512
  62.1045    79.2495   84.1787  162.798        77.65    350.013   101.796
  45.1771    52.8014   27.6889   47.4956   …   24.8448   27.3681   28.364
 121.477    107.507    90.5803   76.5274       37.325    46.2384   52.9135
  58.2984    63.9953   32.4621   33.1899       10.8508   54.439    76.9715
 376.803    351.246   125.95     83.358       639.338   124.034   433.208
 139.829     39.0857   94.5039  110.321       188.464    25.5023  166.945
   ⋮                                       ⋱                      
   7.93469   27.5509   15.3937    7.64014      15.722    16.1983   22.3604
 256.209     21.0913   96.6817  164.891        52.7352   46.4543  185.486
 111.957     97.4

In [8]:
Menergy = Matrix(energy);
Menergy = Menergy[:,2:21]

1000×20 Matrix{Float64}:
 58.4516  56.4218  53.3182  61.1425  …  58.5737  47.434   60.5322  53.7255
 47.1307  49.9741  49.4615  53.4576     45.5567  44.2967  43.7656  51.6316
 72.7165  64.8131  62.9752  76.3534     59.1579  56.775   61.0008  60.1781
 44.5003  51.6669  45.6587  44.7678     55.7139  41.9219  39.5938  45.9446
 48.9771  41.3224  54.4063  53.2036     57.0393  53.6506  41.1886  56.2313
 14.4386  19.9577  18.9604  21.8064  …  18.9657  18.891   20.1127  18.9616
 72.9008  66.1734  71.4013  64.5653     67.8859  62.7328  65.5753  59.0646
 31.1955  29.6268  25.2512  26.9403     32.0769  27.4883  28.1615  26.8298
 73.8362  79.5029  72.8443  83.7081     78.4482  65.0861  78.7875  77.2289
 74.7685  82.2525  71.9211  75.4779     70.681   75.8865  76.0492  71.2974
  ⋮                                  ⋱                             
 27.0395  20.4201  21.4507  20.7178     24.9886  20.9023  22.0435  24.2558
 32.4131  23.5574  29.1081  27.2125     25.1626  28.5859  23.9509  24.7429
 49.252

In [21]:
model = Model(Gurobi.Optimizer)
@variable(model, X[1:1000, 1:20]>=0);
@objective(model, Min, sum(sum(Menergy[i,j]*X[i,j] for j in 1:20) for i in 1:1000));
@constraint(model,a_constraint[i in 1:1000 ,j in 1:20],X[i,j] >= 0 );
@constraint(model,b_constraint[i in 1:1000],sum(X[i,j] for j in 1:20) == 1 );
@constraint(model,c_constraint[j in 1:20],sum(Mutilization[i,j]*X[i,j] for i in 1:1000) <= Mmaxutil[j] );

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-01


In [22]:
#println(model)

In [23]:
optimize!(model)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 21020 rows, 20000 columns and 60000 nonzeros
Model fingerprint: 0xd1c00d75
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [7e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+03]
Presolve removed 20000 rows and 0 columns
Presolve time: 0.35s
Presolved: 1020 rows, 20000 columns, 40000 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.000e+04
 Factor NZ  : 2.168e+04 (roughly 9 MB of memory)
 Factor Ops : 4.729e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.27490183e+06  0.00000000e+00  

In [24]:
objective_value(model)

46693.57908506735

0.0
1.2505552149377763e-12


1000.0


In [14]:
println(stderr, "hello!")

hello!


In [36]:
i = 1

for j in 1:20
    println(value(X[i,j]))
end

0.0


UndefVarError: UndefVarError: X1 not defined

In [29]:
model2 = Model(Gurobi.Optimizer)
@variable(model2, X2[1:1000, 1:20]>=0);
@objective(model2, Min, sum(sum(Menergy[i,j]*X2[i,j] for j in 1:20) for i in 1:1000));
@constraint(model2,a_constraint[i in 1:1000 ,j in 1:20],X2[i,j] >= 0 );
@constraint(model2,b_constraint[i in 1:1000],sum(X2[i,j] for j in 1:20) == 1 );
#@constraint(model2,c_constraint[j in 1:20],sum(Mutilization[i,j]*X2[i,j] for i in 1:1000) <= Mmaxutil[j] );

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-01


In [30]:
optimize!(model2)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 21000 rows, 20000 columns and 40000 nonzeros
Model fingerprint: 0xd940943e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 21000 rows and 20000 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0883161e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds (0.01 work units)
Optimal objective  4.088316111e+04

User-callback calls 38, time in user-callback 0.00 sec


In [35]:

cnt = 0
for i in 1:1000
    temp = 0
    for j in 1:20
        temp += abs(value(X[i,j]) - value(X2[i,j]))
    end
    if temp > 0.001
        println(i)
        cnt += 1
    end
end

println(cnt)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


19
20
21
22
23
24
25
26
28
29
30
31
32
33
34
35
37
38
39
40
41
42
43
44
45
46
47
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
67
68
69
70
71
72
73
74
75
76
77


78
79
80
81
82
83
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
104
105
106
107
108
109
110
111
113
114
115
116
117
118
119
120
121
123
124
125
126
127
128
129
130
131
132
133
134
136
137
138
139
140
141
142
143
145
147
148
149
150
151
152
153
155
157
158
159


160
161
162
163
164
165
166
167
168
169
171
172
173
174
175
177
178
179
180
181
182
183
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
212
213
214
215
217
219
220
221
222
223
224
225
226
227
230
231
232
234
235
236
237
239
240
241
242
243
244
245
246
247
249
250
251
252
253
254
255
256
257
258
260


261
262
263
264
265
266
267
268
269
271
272
273
274
275
276
277
278
279
280
281
282
283
284
286
287
288
289
290
291
292
294
296
297
298
299
300
301
302
303
304
306
307
308
309
310
311
312
313
314
315
317
318
319
321
322
323
324
325
326
327
328
329
331
332
333
334
335
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
377
378
379
380
381
382
383
384
386
387


388
389
390


391
392
393
394
395
396
397
398
399
401
403
404
405
406
407
408
409
410
411
412
413
414
416
417
418
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
436
438
439
441
442
443
444
445
447
448
449
451
453
454
455
456
458
459
460
461
462
463
464
465
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
511
512
513
514
515
516
517
518
519
521
522
523
524
525
526
527
528
530
531
532
533
534
535
536
537
538
540
541
542
543
546
548
549
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
574
575
576
577
578
579
580
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
605
607
608
609
611
612
613
614
615
616
617
618
619
620
621
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
651
654
655
656
657
660
661
662
663
664
665
667
668
669
670
671
672
673


906
907
908
909
910
911
912
913
914
915
916
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
942
944
945
946
947
948
949
950
951
952
953
954
956
957
959
961
963
964
965
966
967
968
969
970
971
972
973
975
976
977
980
981
982
984
985
986
987
988
989
991
992
993
994
995
997
998
999
1000
885


In [41]:
i = 3

for j in 1:20
    println(value(X[i,j])-value(X2[i,j]))
end

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
